# 추천 시스템 (Recommender Systems)

* 추천 시스템은 크게 두가지로 구분 가능
  * 컨텐츠 기반 필터링 (content-based filtering)
  * 협업 필터링 (collaborative filtering)
* 두가지를 조합한 hybrid 방식도 가능
* 컨텐츠 기반 필터링은 지금까지 사용자의 이전 행동과 명시적 피드백을 통해 사용자가 좋아하는 것과 유사한 항목을 추천
* 협업 필터링은 사용자와 항목간의 유사성을 동시에 사용해 추천

## Surprise

* 추천 시스템 개발을 위한 라이브러리
* 다양한 모델과 데이터 제공
* scikit-learn과 유사한 사용 방법

간단한 surprise 실습

In [2]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

In [3]:
data = Dataset.load_builtin('ml-100k', prompt=False)
data.raw_ratings[:10]

# 196 사용자가, 242 영화에, 3점 평점을 부여

Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /Users/ihyeongseob/.surprise_data/ml-100k


[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

In [4]:
# 예측을 하기 위한 model
model = SVD()

In [5]:
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9368  0.9340  0.9454  0.9352  0.9364  0.9376  0.0040  
MAE (testset)     0.7369  0.7356  0.7419  0.7411  0.7379  0.7387  0.0024  
Fit time          0.41    0.41    0.37    0.37    0.37    0.39    0.02    
Test time         0.05    0.04    0.06    0.06    0.06    0.06    0.01    


{'test_rmse': array([0.9368317 , 0.93402712, 0.94539927, 0.93523186, 0.93636427]),
 'test_mae': array([0.73687746, 0.73556402, 0.74189964, 0.74111626, 0.73787729]),
 'fit_time': (0.40581393241882324,
  0.4089529514312744,
  0.37186193466186523,
  0.37387871742248535,
  0.3706223964691162),
 'test_time': (0.04619002342224121,
  0.04347515106201172,
  0.06258320808410645,
  0.06483936309814453,
  0.06422305107116699)}

## 컨텐츠 기반 필터링 (Content-based Filtering)

* 컨텐츠 기반 필터링은 `이전의 행동과 명시적 피드백을 통해 좋아하는 것과 유사한 항목을 추천`
  * ex) 내가 지금 까지 시청한 영화 목록과 다른 사용자의 시청 목록을 비교해 나와 비슷한 취향의 사용자가 시청한 영화를 추천
* 유사도를 기반으로 추천
* 컨텐츠 기반 필터링은 다음과 같은 장단점이 있다.
  * 장점
    * 많은 수의 사용자를 대상으로 쉽게 확장 가능
    * 사용자가 관심을 갖지 않던 상품 추천 가능
  * 단점
    * 입력 특성을 직접 설계해야 하기 때문에 많은 도메인 지식이 필요
    * 사용자의 기존 관심사항을 기반으로만 추천 가능

In [8]:
import numpy as np
from surprise import Dataset

In [9]:
data = Dataset.load_builtin('ml-100k', prompt=False)
raw_data = np.array(data.raw_ratings, dtype=int)

In [10]:
raw_data[:, 0] -= 1
raw_data[:, 1] -= 1

In [12]:
n_users = np.max(raw_data[:, 0]) # 총 user수
n_movies = np.max(raw_data[:, 1]) # 총 movie수
shape = (n_users+1, n_movies+1)
shape

(943, 1682)

In [13]:
adj_matrix = np.ndarray(shape, dtype=int)
for user_id, movie_id, rating, time in raw_data :
    adj_matrix[user_id][movie_id] = 1.  # 인접행렬 -> 1이 있는 위치가 데이터가 있는 부분
adj_matrix

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [14]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix) :
    if my_id != user_id :
        similarity = np.dot(my_vector, user_vector)
        if similarity > best_match :
            best_match = similarity
            best_match_id = user_id
            best_match_vector = user_vector
print("Best Match : {}, Best Match ID : {}" .format(best_match, best_match_id))

Best Match : 183, Best Match ID : 275


In [15]:
recommend_list = [] 
for i, log in enumerate(zip(my_vector, best_match_vector)) :
    log1, log2 = log 
    if log1 < 1. and log2 > 0. :
        recommend_list.append(i)
print(recommend_list)

[272, 273, 275, 280, 281, 283, 287, 288, 289, 290, 292, 293, 297, 299, 300, 301, 302, 306, 312, 314, 315, 316, 317, 321, 322, 323, 324, 327, 330, 331, 332, 333, 339, 342, 345, 346, 353, 354, 355, 356, 357, 363, 364, 365, 366, 372, 374, 378, 379, 381, 382, 383, 384, 385, 386, 387, 390, 391, 392, 394, 395, 396, 398, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 412, 414, 416, 417, 418, 419, 420, 422, 424, 425, 426, 427, 428, 430, 431, 432, 435, 442, 446, 447, 448, 449, 450, 451, 452, 454, 455, 457, 460, 461, 462, 468, 469, 470, 471, 472, 473, 474, 478, 495, 500, 507, 517, 522, 525, 530, 539, 540, 543, 545, 546, 548, 549, 550, 551, 553, 557, 558, 560, 561, 562, 563, 565, 566, 567, 568, 570, 571, 574, 575, 576, 577, 580, 581, 582, 585, 587, 589, 590, 594, 596, 602, 623, 626, 627, 630, 633, 635, 639, 646, 648, 651, 652, 654, 657, 664, 668, 671, 677, 678, 681, 683, 684, 685, 690, 691, 692, 695, 696, 708, 709, 714, 718, 719, 720, 724, 726, 727, 731, 733, 734, 736, 738, 741, 742, 745,

### 유클리드 거리를 사용해 추천
$$euclidean = \sqrt{\sum_{d=1}^{D}(A_i - B_i)^2}$$
* 거리가 가까울 수록(값이 작을 수록) 나와 유사한 사용자

In [17]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999, -1, []

for user_id, user_vector in enumerate(adj_matrix) :
    if my_id != user_id :
        euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
        if euclidean_dist < best_match :
            best_match = euclidean_dist
            best_match_id = user_id
            best_match_vector = user_vector
print("Best Match : {}, Best Match ID : {}" .format(best_match, best_match_id))

Best Match : 14.832396974191326, Best Match ID : 737


In [18]:
recommend_list = [] 
for i, log in enumerate(zip(my_vector, best_match_vector)) :
    log1, log2 = log 
    if log1 < 1. and log2 > 0. :
        recommend_list.append(i)
print(recommend_list)

[297, 312, 317, 342, 356, 366, 379, 384, 392, 402, 404, 407, 417, 422, 428, 433, 448, 454, 469, 473, 495, 510, 516, 526, 527, 549, 567, 602, 635, 649, 650, 654, 658, 661, 664, 696, 731, 746, 750, 754, 915, 918, 925, 929, 950, 968, 1015, 1046]


### 코사인 유사도를 사용해 추천

\begin{equation}
cos \theta = \frac{A \cdot B}{||A|| \times ||B||}
\end{equation}
* 두 벡터가 이루고 있는 각을 계산

In [19]:
def compute_cos_similarity(v1, v2)  :
    norm1 = np.sqrt(np.sum(np.square(v1)))
    norm2 = np.sqrt(np.sum(np.square(v2)))
    dot = np.dot(v1, v2)
    return dot / (norm1 * norm2)

In [20]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix) :
    if my_id != user_id :
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match :
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector
print("Best Match : {}, Best Match ID : {}" .format(best_match, best_match_id))

Best Match : 0.5278586163659506, Best Match ID : 915


In [21]:
recommend_list = [] 
for i, log in enumerate(zip(my_vector, best_match_vector)) :
    log1, log2 = log 
    if log1 < 1. and log2 > 0. :
        recommend_list.append(i)
print(recommend_list)

# 915번 user는 봤지만 나는 안 본 영화들 추천

[272, 275, 279, 280, 283, 285, 289, 294, 297, 316, 317, 355, 365, 366, 368, 379, 380, 381, 384, 386, 392, 398, 401, 404, 416, 420, 422, 424, 426, 427, 430, 432, 450, 460, 461, 466, 469, 471, 473, 474, 475, 479, 482, 483, 497, 505, 508, 510, 511, 522, 526, 527, 529, 530, 534, 536, 540, 545, 548, 549, 556, 557, 558, 560, 565, 567, 568, 569, 577, 580, 581, 582, 592, 596, 630, 635, 639, 641, 649, 651, 654, 673, 677, 678, 683, 684, 692, 696, 701, 703, 707, 708, 709, 712, 714, 719, 720, 726, 731, 734, 736, 738, 740, 745, 747, 754, 755, 761, 762, 763, 766, 780, 789, 791, 805, 819, 823, 824, 830, 843, 862, 865, 918, 929, 930, 938, 942, 943, 947, 958, 959, 960, 970, 977, 1004, 1008, 1009, 1010, 1013, 1041, 1045, 1069, 1072, 1073, 1078, 1097, 1100, 1108, 1112, 1118, 1134, 1193, 1205, 1207, 1216, 1219, 1267, 1334, 1400, 1427, 1596, 1681]


### 기존 방법에 명시적 피드백(사용자가 평가한 영화 점수)을 추가해 실험

In [29]:
adj_matrix = np.ndarray(shape, dtype=int)
for user_id, movie_id, rating, time in raw_data :
    adj_matrix[user_id][movie_id] = rating  # 1,0(봤냐 안봤냐)이 아닌 평점 정보로 채움
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

#### 유클리드 거리를 사용해 추천
$$euclidean = \sqrt{\sum_{d=1}^{D}(A_i - B_i)^2}$$
* 거리가 가까울 수록(값이 작을 수록) 나와 유사한 사용자

In [30]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999, -1, []

for user_id, user_vector in enumerate(adj_matrix) :
    if my_id != user_id :
        euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
        if euclidean_dist < best_match :
            best_match = euclidean_dist
            best_match_id = user_id
            best_match_vector = user_vector
print("Best Match : {}, Best Match ID : {}" .format(best_match, best_match_id))

Best Match : 55.06359959174482, Best Match ID : 737


In [31]:
recommend_list = [] 
for i, log in enumerate(zip(my_vector, best_match_vector)) :
    log1, log2 = log 
    if log1 < 1. and log2 > 0. :
        recommend_list.append(i)
print(recommend_list)

[297, 312, 317, 342, 356, 366, 379, 384, 392, 402, 404, 407, 417, 422, 428, 433, 448, 454, 469, 473, 495, 510, 516, 526, 527, 549, 567, 602, 635, 649, 650, 654, 658, 661, 664, 696, 731, 746, 750, 754, 915, 918, 925, 929, 950, 968, 1015, 1046]


#### 코사인 유사도를 사용해 추천

\begin{equation}
cos \theta = \frac{A \cdot B}{||A|| \times ||B||}
\end{equation}
* 두 벡터가 이루고 있는 각을 계산

In [32]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix) :
    if my_id != user_id :
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match :
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector
print("Best Match : {}, Best Match ID : {}" .format(best_match, best_match_id))

Best Match : 0.569065731527988, Best Match ID : 915


In [33]:
recommend_list = [] 
for i, log in enumerate(zip(my_vector, best_match_vector)) :
    log1, log2 = log 
    if log1 < 1. and log2 > 0. :
        recommend_list.append(i)
print(recommend_list)

# 915번 user는 봤지만 나는 안 본 영화들 추천

[272, 275, 279, 280, 283, 285, 289, 294, 297, 316, 317, 355, 365, 366, 368, 379, 380, 381, 384, 386, 392, 398, 401, 404, 416, 420, 422, 424, 426, 427, 430, 432, 450, 460, 461, 466, 469, 471, 473, 474, 475, 479, 482, 483, 497, 505, 508, 510, 511, 522, 526, 527, 529, 530, 534, 536, 540, 545, 548, 549, 556, 557, 558, 560, 565, 567, 568, 569, 577, 580, 581, 582, 592, 596, 630, 635, 639, 641, 649, 651, 654, 673, 677, 678, 683, 684, 692, 696, 701, 703, 707, 708, 709, 712, 714, 719, 720, 726, 731, 734, 736, 738, 740, 745, 747, 754, 755, 761, 762, 763, 766, 780, 789, 791, 805, 819, 823, 824, 830, 843, 862, 865, 918, 929, 930, 938, 942, 943, 947, 958, 959, 960, 970, 977, 1004, 1008, 1009, 1010, 1013, 1041, 1045, 1069, 1072, 1073, 1078, 1097, 1100, 1108, 1112, 1118, 1134, 1193, 1205, 1207, 1216, 1219, 1267, 1334, 1400, 1427, 1596, 1681]


## 협업 필터링(Collaborative Filtering)

* 사용자와 항목의 유사성을 동시에 고려해 추천
* 기존에 내 관심사가 아닌 항목이라도 추천 가능
* 자동으로 임베딩 학습 가능
* 협업 필터링은 다음과 같은 장단점을 갖고 있다.
  * 장점
    * 자동으로 임베딩을 학습하기 때문에 도메인 지식이 필요 없다.
    * 기존의 관심사가 아니더라도 추천 가능
  * 단점
    * 학습 과정에 나오지 않은 항목은 임베딩을 만들 수 없음
    * 추가 특성을 사용하기 어려움

In [34]:
from surprise import KNNBasic, SVD, SVDpp, NMF 
from surprise import Dataset
from surprise.model_selection import cross_validate


In [35]:
data = Dataset.load_builtin('ml-100k', prompt=False)

### KNN을 사용한 협업 필터링

In [36]:
model = KNNBasic()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

Computing the msd similarity matrix...
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9746  0.9836  0.9768  0.9786  0.9788  0.9785  0.0030  
MAE (testset)     0.7703  0.7789  0.7705  0.7703  0.7731  0.7726  0.0033  
Fit time          0.17    0.17    0.14    0.13    0.10    0.14    0.03    
Test time         0.96    0.96    0.96    0.96    0.90    0.95    0.02    


{'test_rmse': array([0.97458487, 0.98355615, 0.97676776, 0.9786208 , 0.97881917]),
 'test_mae': array([0.77025058, 0.77894262, 0.77052348, 0.77027259, 0.77309148]),
 'fit_time': (0.1748349666595459,
  0.17022323608398438,
  0.14327716827392578,
  0.12645411491394043,
  0.09568405151367188),
 'test_time': (0.9616110324859619,
  0.9573972225189209,
  0.9634521007537842,
  0.956934928894043,
  0.9036569595336914)}

### SVD을 사용한 협업 필터링

In [37]:
model = SVD()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9327  0.9380  0.9390  0.9350  0.9338  0.9357  0.0024  
MAE (testset)     0.7338  0.7374  0.7432  0.7389  0.7349  0.7376  0.0033  
Fit time          0.35    0.34    0.35    0.36    0.33    0.35    0.01    
Test time         0.07    0.06    0.06    0.06    0.05    0.06    0.00    


{'test_rmse': array([0.93265512, 0.93796332, 0.939021  , 0.93502514, 0.93382743]),
 'test_mae': array([0.73382897, 0.73741255, 0.74315762, 0.73892011, 0.73490141]),
 'fit_time': (0.35153913497924805,
  0.3381078243255615,
  0.3535301685333252,
  0.35639190673828125,
  0.32724499702453613),
 'test_time': (0.06630921363830566,
  0.06447410583496094,
  0.06135296821594238,
  0.06233024597167969,
  0.05318021774291992)}

### NMF를 사용한 협업 필터링

In [38]:
model = NMF()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9673  0.9603  0.9670  0.9467  0.9738  0.9630  0.0092  
MAE (testset)     0.7587  0.7536  0.7609  0.7437  0.7646  0.7563  0.0072  
Fit time          0.44    0.45    0.45    0.44    0.41    0.44    0.01    
Test time         0.06    0.06    0.06    0.06    0.05    0.06    0.01    


{'test_rmse': array([0.96725126, 0.96026959, 0.9669786 , 0.94672275, 0.97375511]),
 'test_mae': array([0.75869025, 0.75358336, 0.76086325, 0.74372208, 0.76461247]),
 'fit_time': (0.43901586532592773,
  0.4486110210418701,
  0.44965076446533203,
  0.44461703300476074,
  0.4098701477050781),
 'test_time': (0.0587000846862793,
  0.05674409866333008,
  0.060236215591430664,
  0.062050819396972656,
  0.045356035232543945)}

### SVD++를 사용한 협업 필터링

In [39]:
model = SVDpp()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9164  0.9179  0.9201  0.9145  0.9223  0.9183  0.0027  
MAE (testset)     0.7166  0.7216  0.7222  0.7182  0.7221  0.7202  0.0023  
Fit time          6.65    6.68    6.68    6.66    6.04    6.54    0.25    
Test time         1.23    1.22    1.23    1.23    1.09    1.20    0.05    


{'test_rmse': array([0.91642418, 0.91789742, 0.92014427, 0.91451734, 0.92233096]),
 'test_mae': array([0.71664401, 0.72162027, 0.72223973, 0.7181867 , 0.72209293]),
 'fit_time': (6.64653205871582,
  6.682145118713379,
  6.68350887298584,
  6.663583993911743,
  6.0432939529418945),
 'test_time': (1.2275030612945557,
  1.220059871673584,
  1.2287178039550781,
  1.229646921157837,
  1.0909218788146973)}

## 하이브리드(Hybrid)

* 컨텐츠 기반 필터링과 협업 필터링을 조합한 방식
* 많은 하이브리드 방식이 존재
* 실습에서는 협업 필터링으로 임베딩을 학습하고 컨텐츠 기반 필터링으로 유사도 기반 추천을 수행하는 추천 엔진 개발

In [41]:
import numpy as np
from sklearn.decomposition import randomized_svd, non_negative_factorization
from surprise import Dataset

In [70]:
data = Dataset.load_builtin('ml-100k', prompt=False)
raw_data = np.array(data.raw_ratings, dtype=int)
raw_data[:, 0] -= 1
raw_data[:, 1] -= 1

In [71]:
n_users = np.max(raw_data[:, 0])
n_movies = np.max(raw_data[:, 1])
shape = (n_users + 1, n_movies + 1)
shape

(943, 1682)

In [72]:
adj_matrix = np.ndarray(shape, dtype=int)
for user_id, movie_id, rating, time in raw_data :
    adj_matrix[user_id][movie_id] = rating

adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [73]:
U, S, V = randomized_svd(adj_matrix, n_components=2) # User, Similarity, V(item)
S = np.diag(S)

In [74]:
print(U.shape) # 
print(S.shape) # 
print(V.shape) # 

# 행렬 U와 행렬 V 사이에 행렬 S가 있는데,
# 행렬 S는 잠재요인(latent)을 찾을 수 있는 특이값 행렬이 된다.d

(943, 2)
(2, 2)
(2, 1682)


In [75]:
# 행렬 U * S * V
np.matmul(np.matmul(U, S), V)


array([[ 3.91732664e+00,  1.47276646e+00,  7.98262054e-01, ...,
         6.24907453e-04,  1.41100865e-02,  1.36545904e-02],
       [ 1.85777231e+00,  3.96191004e-01,  5.05705266e-01, ...,
         5.38862834e-03,  1.77236848e-03,  5.26947498e-04],
       [ 8.94989194e-01,  1.71578375e-01,  2.51738307e-01, ...,
         2.92094664e-03,  5.39931740e-04, -1.25744254e-04],
       ...,
       [ 9.92051702e-01,  2.10814843e-01,  2.70363019e-01, ...,
         2.89019076e-03,  9.34216584e-04,  2.66601794e-04],
       [ 1.30425384e+00,  5.27669958e-01,  2.50080195e-01, ...,
        -4.20678335e-04,  5.30525887e-03,  5.28070343e-03],
       [ 2.82999405e+00,  9.70812212e-01,  6.15871604e-01, ...,
         2.02091491e-03,  8.67740556e-03,  8.03107394e-03]])

* 사용자 기반 추천
* 나와 비슷한 취향을 가진 다른 사용자의 행동을 추천
* 사용자 특징 벡터의 유사도 사용

In [54]:
# 사용자 기반 추천(행렬 U 사용)
my_id, my_vector = 0, U[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(U) :
    if my_id != user_id :
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match :
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector
print("Best Match : {}, Best Match ID : {}" .format(best_match, best_match_id))


# SVD를 돌린 후 cosine simularity를 돌린 결과 --> 99% (매우 높아짐)

Best Match : 0.9996193522049851, Best Match ID : 685


In [76]:
recommend_list = [] 
for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])) :
    log1, log2 = log 
    if log1 < 1. and log2 > 0. :
        recommend_list.append(i)
print(recommend_list)

# 사용자 기반으로 추천된 목록

[274, 356, 403, 407, 418, 426, 429, 431, 454, 462, 473, 476, 477, 478, 479, 494, 497, 500, 519, 526, 567, 603, 611, 614, 632, 654, 656, 691, 728, 741, 836, 928, 967, 968]


* 항목 기반 추천
* 내가 본 항목과 비슷한 항목을 추천
* 항목 특징 벡터의 유사도 사용

In [77]:
# 항목 기반 추천(행렬 V 사용)
my_id, my_vector = 0, V.T[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(V.T) :
    if my_id != user_id :
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match :
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector
print("Best Match : {}, Best Match ID : {}" .format(best_match, best_match_id))


# SVD를 돌린 후 cosine simularity를 돌린 결과 --> 99% (매우 높아짐)

Best Match : 0.9999999949256824, Best Match ID : 1287


In [59]:
recommend_list = [] 
for i, user_vector in enumerate(adj_matrix) :
    if adj_matrix[i][my_id] > 0.9 :
        recommend_list.append(i)
print(recommend_list)

# 항목 기반으로 추천된 목록

[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 37, 40, 41, 42, 43, 44, 48, 53, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 69, 71, 72, 74, 76, 78, 80, 81, 82, 83, 88, 91, 92, 93, 94, 95, 96, 98, 100, 101, 105, 107, 108, 116, 119, 120, 123, 124, 127, 129, 130, 133, 136, 137, 140, 143, 144, 147, 149, 150, 156, 157, 159, 161, 167, 173, 176, 177, 180, 181, 183, 188, 192, 193, 197, 198, 199, 200, 201, 202, 203, 208, 209, 212, 215, 221, 222, 229, 230, 231, 233, 234, 235, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 255, 261, 262, 264, 267, 270, 272, 273, 274, 275, 276, 278, 279, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 300, 302, 304, 306, 307, 310, 311, 312, 313, 319, 321, 323, 324, 325, 326, 329, 330, 331, 335, 337, 338, 339, 342, 343, 344, 346, 347, 349, 356, 358, 359, 362, 364, 370, 373, 377, 378, 379, 380, 386, 387, 388, 389, 392, 393, 394, 395, 397, 398, 400, 401, 402, 405, 406, 410, 411, 415, 416, 418, 421, 423, 424, 428, 431, 433, 434, 437, 

## 비음수 행렬 분해를 사용한 하이브리드 추천

In [78]:
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [79]:
A, B, iter = non_negative_factorization(adj_matrix, n_components=2)

In [80]:
np.matmul(A, B)

array([[3.71107433e+00, 1.48461856e+00, 7.39541570e-01, ...,
        3.64501983e-03, 1.45513751e-02, 1.44116215e-02],
       [2.11729713e+00, 2.37145679e-01, 5.51637757e-01, ...,
        4.76290749e-03, 2.84605930e-05, 0.00000000e+00],
       [9.85325089e-01, 1.10360320e-01, 2.56715279e-01, ...,
        2.21651094e-03, 1.32446863e-05, 0.00000000e+00],
       ...,
       [1.04478344e+00, 1.17019891e-01, 2.72206478e-01, ...,
        2.35026384e-03, 1.40439223e-05, 0.00000000e+00],
       [1.45769331e+00, 5.42108391e-01, 2.99217251e-01, ...,
        1.61232500e-03, 5.15892655e-03, 5.10748255e-03],
       [2.44709957e+00, 9.41278705e-01, 4.95671746e-01, ...,
        2.56934867e-03, 9.08400301e-03, 8.99501717e-03]])

* 사용자 기반 추천
* 나와 비슷한 취향을 가진 다른 사용자의 행동을 추천
* 사용자 특징 벡터의 유사도 사용

In [81]:
# 사용자 기반 추천(행렬 U 사용)
my_id, my_vector = 0, U[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(U) :
    if my_id != user_id :
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match :
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector
print("Best Match : {}, Best Match ID : {}" .format(best_match, best_match_id))

Best Match : 0.9999942297104878, Best Match ID : 235


In [82]:
recommend_list = [] 
for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])) :
    log1, log2 = log 
    if log1 < 1. and log2 > 0. :
        recommend_list.append(i)
print(recommend_list)

# 사용자 기반으로 추천된 목록

[272, 273, 274, 281, 285, 288, 293, 297, 303, 306, 312, 317, 327, 332, 369, 410, 418, 419, 422, 426, 428, 431, 434, 442, 461, 475, 477, 482, 495, 503, 504, 505, 506, 509, 519, 520, 522, 525, 531, 545, 548, 590, 594, 595, 613, 631, 654, 658, 660, 672, 684, 685, 691, 695, 698, 704, 716, 728, 734, 749, 755, 863, 865, 933, 1012, 1038, 1101, 1327, 1400]


* 항목 기반 추천
* 내가 본 항목과 비슷한 항목을 추천
* 항목 특징 벡터의 유사도 사용

In [83]:
# 항목 기반 추천(행렬 V 사용)
my_id, my_vector = 0, V.T[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(V.T) :
    if my_id != user_id :
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match :
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector
print("Best Match : {}, Best Match ID : {}" .format(best_match, best_match_id))

Best Match : 0.9999999949256824, Best Match ID : 1287


In [ ]:
recommend_list = [] 
for i, user_vector in enumerate(adj_matrix) :
    if adj_matrix[i][my_id] > 0.9 :
        recommend_list.append(i)
print(recommend_list)

# 항목 기반으로 추천된 목록

[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 37, 40, 41, 42, 43, 44, 48, 53, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 69, 71, 72, 74, 76, 78, 80, 81, 82, 83, 88, 91, 92, 93, 94, 95, 96, 98, 100, 101, 105, 107, 108, 116, 119, 120, 123, 124, 127, 129, 130, 133, 136, 137, 140, 143, 144, 147, 149, 150, 156, 157, 159, 161, 167, 173, 176, 177, 180, 181, 183, 188, 192, 193, 197, 198, 199, 200, 201, 202, 203, 208, 209, 212, 215, 221, 222, 229, 230, 231, 233, 234, 235, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 255, 261, 262, 264, 267, 270, 272, 273, 274, 275, 276, 278, 279, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 300, 302, 304, 306, 307, 310, 311, 312, 313, 319, 321, 323, 324, 325, 326, 329, 330, 331, 335, 337, 338, 339, 342, 343, 344, 346, 347, 349, 356, 358, 359, 362, 364, 370, 373, 377, 378, 379, 380, 386, 387, 388, 389, 392, 393, 394, 395, 397, 398, 400, 401, 402, 405, 406, 410, 411, 415, 416, 418, 421, 423, 424, 428, 431, 433, 434, 437, 